In [140]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier


from sklearn.ensemble import GradientBoostingClassifier

In [141]:
#import des donnees

path='C:\\Users\\oussa\\Downloads\\Data Science\\Master Statistique Big Data Dauphine\\Module 3\\Mini-Projet\\'
data_train=pd.read_csv(path+'input_train.csv')
train_output=pd.read_csv(path+'challenge_fichier_de_sortie_dentrainement_prevision_du_statut_des_reclamations_liees_aux_transactions.csv',sep=';')
data_train=pd.merge(data_train,train_output,left_on='ID',right_on='ID',how='inner')
data_train.head()


,ID,SHIPPING_MODE,SHIPPING_PRICE,WARRANTIES_FLG,WARRANTIES_PRICE,CARD_PAYMENT,COUPON_PAYMENT,RSP_PAYMENT,WALLET_PAYMENT,PRICECLUB_STATUS,...,BUYER_DEPARTMENT,BUYING_DATE,SELLER_SCORE_COUNT,SELLER_SCORE_AVERAGE,SELLER_COUNTRY,SELLER_DEPARTMENT,PRODUCT_TYPE,PRODUCT_FAMILY,ITEM_PRICE,CLAIM_TYPE
0,0,NORMAL,NaN,False,NaN,1,0,1,0,UNSUBSCRIBED,...,34,3/2017,10000<100000,46.0,"FRANCE, METROPOLITAN",61,CELLPHONE_ACCESSORY,ELECTRONICS,<10,DAMAGED
1,1,NORMAL,NaN,False,NaN,1,0,0,0,UNSUBSCRIBED,...,77,8/2017,10000<100000,45.0,"FRANCE, METROPOLITAN",30,CELLPHONE_ACCESSORY,ELECTRONICS,<10,-
2,2,NORMAL,NaN,False,NaN,0,0,0,1,PLATINUM,...,58,5/2017,10000<100000,43.0,CHINA,-1,TOYS,BABY,<10,NOT_RECEIVED
3,3,RECOMMANDE,NaN,True,5<20,1,0,0,0,UNSUBSCRIBED,...,31,5/2017,10000<100000,44.0,"FRANCE, METROPOLITAN",2,GARDEN_TOOLS,WHITE,50<100,-
4,4,RECOMMANDE,NaN,False,NaN,1,0,1,0,PLATINUM,...,93,9/2017,1000<10000,44.0,CHINA,-1,MODEL,BABY,1000<5000,WITHDRAWAL


In [142]:
data_train.shape

(100000, 23)

In [143]:
data_train.CLAIM_TYPE.value_counts()/len(data_train.CLAIM_TYPE)#ratio des classes en presence

-                           0.49977
NOT_RECEIVED                0.14808
SELLER_CANCEL_POSTERIORI    0.13782
WITHDRAWAL                  0.07001
DAMAGED                     0.05922
DIFFERENT                   0.04279
UNDEFINED                   0.04066
FAKE                        0.00165
Name: CLAIM_TYPE, dtype: float64

In [144]:
list(data_train)

['ID',
 'SHIPPING_MODE',
 'SHIPPING_PRICE',
 'WARRANTIES_FLG',
 'WARRANTIES_PRICE',
 'CARD_PAYMENT',
 'COUPON_PAYMENT',
 'RSP_PAYMENT',
 'WALLET_PAYMENT',
 'PRICECLUB_STATUS',
 'REGISTRATION_DATE',
 'PURCHASE_COUNT',
 'BUYER_BIRTHDAY_DATE',
 'BUYER_DEPARTMENT',
 'BUYING_DATE',
 'SELLER_SCORE_COUNT',
 'SELLER_SCORE_AVERAGE',
 'SELLER_COUNTRY',
 'SELLER_DEPARTMENT',
 'PRODUCT_TYPE',
 'PRODUCT_FAMILY',
 'ITEM_PRICE',
 'CLAIM_TYPE']

In [145]:
data_train.dtypes

ID                        int64
SHIPPING_MODE            object
SHIPPING_PRICE           object
WARRANTIES_FLG             bool
WARRANTIES_PRICE         object
CARD_PAYMENT              int64
COUPON_PAYMENT            int64
RSP_PAYMENT               int64
WALLET_PAYMENT            int64
PRICECLUB_STATUS         object
REGISTRATION_DATE         int64
PURCHASE_COUNT           object
BUYER_BIRTHDAY_DATE     float64
BUYER_DEPARTMENT          int64
BUYING_DATE              object
SELLER_SCORE_COUNT       object
SELLER_SCORE_AVERAGE    float64
SELLER_COUNTRY           object
SELLER_DEPARTMENT         int64
PRODUCT_TYPE             object
PRODUCT_FAMILY           object
ITEM_PRICE               object
CLAIM_TYPE               object
dtype: object

In [146]:
for i in list(data_train):
    print(i,data_train[i].unique())

ID [    0     1     2 ... 99997 99998 99999]
SHIPPING_MODE ['NORMAL' 'RECOMMANDE' 'EXPRESS_DELIVERY' 'SUIVI' 'SO_RECOMMANDE'
 'MONDIAL_RELAY' 'MONDIAL_RELAY_PREPAYE' 'SO_POINT_RELAIS' nan
 'CHRONOPOST' 'PICKUP' 'Kiala']
SHIPPING_PRICE [nan '5<10' '10<20' '<1' '1<5' '>20']
WARRANTIES_FLG [False  True]
WARRANTIES_PRICE [nan '5<20' '<5' '20<50' '50<100' '100<500']
CARD_PAYMENT [1 0]
COUPON_PAYMENT [0 1]
RSP_PAYMENT [1 0]
WALLET_PAYMENT [0 1]
PRICECLUB_STATUS ['UNSUBSCRIBED' 'PLATINUM' 'SILVER' 'REGULAR' 'GOLD' nan]
REGISTRATION_DATE [2015 2013 2007 2010 2005 2016 2004 2017 2012 2006 2009 2002 2014 2008
 2011 2003 2001]
PURCHASE_COUNT ['<5' '50<100' '>500' '5<20' '100<500' '20<50']
BUYER_BIRTHDAY_DATE [1992. 1952. 1991. 1955. 1984. 1987. 1986.   nan 1960. 1993. 1978. 1971.
 1990. 1983. 1973. 1985. 1988. 1994. 1998. 1970. 1956. 1961. 1974. 1989.
 1976. 1981. 1982. 1966. 1959. 2000. 1979. 1962. 1968. 1945. 1977. 1969.
 1980. 1954. 1957. 1997. 1995. 1996. 1958. 1972. 1946. 1963. 1948. 1965.
 

In [147]:
data_test=pd.read_csv(path+'input_test.csv')

In [148]:
data_train['Train_Test']='train'
data_test['Train_Test']='test'

In [149]:
data_global=pd.concat([data_train.drop(['CLAIM_TYPE','ID'],axis=1),data_test.drop(['ID'],axis=1)],axis=0)

In [150]:

#on convertir toutes les variables en variables categorielle, les na sont considerees comme des categories
data_global.WARRANTIES_FLG=data_global.WARRANTIES_FLG.astype(int)
data_global.WARRANTIES_FLG=data_global.WARRANTIES_FLG.astype(object)
data_global.BUYER_DEPARTMENT=data_global.BUYER_DEPARTMENT.astype(object)
data_global.SELLER_DEPARTMENT=data_global.SELLER_DEPARTMENT.astype(object)

data_global.SELLER_SCORE_AVERAGE=data_global.SELLER_SCORE_AVERAGE.astype(object)

data_global.BUYER_BIRTHDAY_DATE=data_global.BUYER_BIRTHDAY_DATE.astype(object)

data_global.REGISTRATION_DATE=data_global.REGISTRATION_DATE.astype(object)


data_global=pd.concat([pd.get_dummies(data_global.drop(['Train_Test'],axis=1),dummy_na=True),data_global.Train_Test],axis=1)

In [151]:
data_global.head()

,CARD_PAYMENT,COUPON_PAYMENT,RSP_PAYMENT,WALLET_PAYMENT,SHIPPING_MODE_CHRONOPOST,SHIPPING_MODE_EXPRESS_DELIVERY,SHIPPING_MODE_Kiala,SHIPPING_MODE_MONDIAL_RELAY,SHIPPING_MODE_MONDIAL_RELAY_PREPAYE,SHIPPING_MODE_NORMAL,...,ITEM_PRICE_1000<5000,ITEM_PRICE_100<500,ITEM_PRICE_10<20,ITEM_PRICE_20<50,ITEM_PRICE_500<1000,ITEM_PRICE_50<100,ITEM_PRICE_<10,ITEM_PRICE_>5000,ITEM_PRICE_nan,Train_Test
0,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,train
1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,train
2,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,train
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,train
4,1,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,train


In [152]:
data_train_preprocessed=pd.concat([data_global[data_global.Train_Test=="train"].drop(['Train_Test'],axis=1),data_train.CLAIM_TYPE],axis=1)
data_test_preprocessed=data_global[data_global.Train_Test=="test"].drop(['Train_Test'],axis=1)


In [153]:
data_train_preprocessed.head()

,CARD_PAYMENT,COUPON_PAYMENT,RSP_PAYMENT,WALLET_PAYMENT,SHIPPING_MODE_CHRONOPOST,SHIPPING_MODE_EXPRESS_DELIVERY,SHIPPING_MODE_Kiala,SHIPPING_MODE_MONDIAL_RELAY,SHIPPING_MODE_MONDIAL_RELAY_PREPAYE,SHIPPING_MODE_NORMAL,...,ITEM_PRICE_1000<5000,ITEM_PRICE_100<500,ITEM_PRICE_10<20,ITEM_PRICE_20<50,ITEM_PRICE_500<1000,ITEM_PRICE_50<100,ITEM_PRICE_<10,ITEM_PRICE_>5000,ITEM_PRICE_nan,CLAIM_TYPE
0,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,DAMAGED
1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,-
2,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,NOT_RECEIVED
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,-
4,1,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,WITHDRAWAL


In [154]:
data_train_preprocessed["aggregated_class"]=data_train_preprocessed.apply(lambda row : 0 if row['CLAIM_TYPE']=='-' else 1,axis=1)


In [155]:
data_train_preprocessed.aggregated_class.value_counts()/len(data_train_preprocessed.aggregated_class)#ratio des classes en presence

1    0.50023
0    0.49977
Name: aggregated_class, dtype: float64

In [156]:
data_train_claims=data_train_preprocessed[data_train_preprocessed.aggregated_class==1]

In [157]:
data_train_claims.CLAIM_TYPE.value_counts()/len(data_train_claims.CLAIM_TYPE)#ratio des classes en presence

NOT_RECEIVED                0.296024
SELLER_CANCEL_POSTERIORI    0.275513
WITHDRAWAL                  0.139956
DAMAGED                     0.118386
DIFFERENT                   0.085541
UNDEFINED                   0.081283
FAKE                        0.003298
Name: CLAIM_TYPE, dtype: float64

In [158]:
#on convertit y en entiers pour sklearn

le = preprocessing.LabelEncoder()
le.fit(data_train_claims[['CLAIM_TYPE']])
data_train_claims_z=data_train_claims[['CLAIM_TYPE']].copy().apply(le.fit_transform)
data_train_claims_z.head()



C:\Users\oussa\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,CLAIM_TYPE
0,0
2,3
4,6
6,5
7,6


In [159]:

le.inverse_transform(data_train_claims_z.CLAIM_TYPE.values)

C:\Users\oussa\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['DAMAGED', 'NOT_RECEIVED', 'WITHDRAWAL', ...,
       'SELLER_CANCEL_POSTERIORI', 'UNDEFINED', 'WITHDRAWAL'],
      dtype=object)

In [160]:
data_train_preprocessed.head()

,CARD_PAYMENT,COUPON_PAYMENT,RSP_PAYMENT,WALLET_PAYMENT,SHIPPING_MODE_CHRONOPOST,SHIPPING_MODE_EXPRESS_DELIVERY,SHIPPING_MODE_Kiala,SHIPPING_MODE_MONDIAL_RELAY,SHIPPING_MODE_MONDIAL_RELAY_PREPAYE,SHIPPING_MODE_NORMAL,...,ITEM_PRICE_100<500,ITEM_PRICE_10<20,ITEM_PRICE_20<50,ITEM_PRICE_500<1000,ITEM_PRICE_50<100,ITEM_PRICE_<10,ITEM_PRICE_>5000,ITEM_PRICE_nan,CLAIM_TYPE,aggregated_class
0,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,DAMAGED,1
1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,-,0
2,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,NOT_RECEIVED,1
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,-,0
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,WITHDRAWAL,1


In [161]:
X_train=data_train_preprocessed.drop(['CLAIM_TYPE','aggregated_class'],axis=1).values
y_train=data_train_preprocessed.aggregated_class.values

In [162]:
data_train_claims.head()

,CARD_PAYMENT,COUPON_PAYMENT,RSP_PAYMENT,WALLET_PAYMENT,SHIPPING_MODE_CHRONOPOST,SHIPPING_MODE_EXPRESS_DELIVERY,SHIPPING_MODE_Kiala,SHIPPING_MODE_MONDIAL_RELAY,SHIPPING_MODE_MONDIAL_RELAY_PREPAYE,SHIPPING_MODE_NORMAL,...,ITEM_PRICE_100<500,ITEM_PRICE_10<20,ITEM_PRICE_20<50,ITEM_PRICE_500<1000,ITEM_PRICE_50<100,ITEM_PRICE_<10,ITEM_PRICE_>5000,ITEM_PRICE_nan,CLAIM_TYPE,aggregated_class
0,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,DAMAGED,1
2,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,NOT_RECEIVED,1
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,WITHDRAWAL,1
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,UNDEFINED,1
7,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,WITHDRAWAL,1


In [163]:
X_train.shape

(100000, 627)

In [164]:
y_train

array([1, 0, 1, ..., 0, 1, 0], dtype=int64)

In [165]:

modele1=RandomForestClassifier(n_estimators=1510,max_features='sqrt',n_jobs=2)

#modele1=GradientBoostingClassifier(n_estimators=250)
#modele=GradientBoostingClassifier()

#modele=GradientBoostingClassifier(n_estimators=500)

modele1.fit(X_train,y_train)

JoblibMemoryError: JoblibMemoryError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\oussa\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\oussa\Anaconda3\lib\runpy.

In [ ]:
X_train_claims=data_train_claims.drop(['CLAIM_TYPE','aggregated_class'],axis=1).values
z_train=data_train_claims_z.values

In [ ]:
modele2=RandomForestClassifier(n_estimators=1510,max_features='sqrt',n_jobs=2)
#modele=GradientBoostingClassifier()
#modele2=GradientBoostingClassifier(n_estimators=250)
#modele=GradientBoostingClassifier(n_estimators=500)

modele2.fit(X_train_claims,z_train)

In [ ]:
data_test_preprocessed.head()

In [ ]:
data_train_preprocessed.head()

In [ ]:
X_train.shape
data_test_preprocessed.head()

In [ ]:
data_test_preprocessed=data_test_preprocessed.drop(['aggregated_class'],axis=1)

In [ ]:
data_test_preprocessed.head()

In [ ]:
#data_test_preprocessed['aggregated_class']=data_test_preprocessed.apply(lambda row: modele1.predict(np.reshape(row.values,(1,-1)) ), axis=1)
#data_test_preprocessed['aggregated_class']=data_test_preprocessed.apply(lambda row: modele1.predict(row.values[None])[0], axis=1)
data_test_preprocessed['aggregated_class']=modele1.predict(data_test_preprocessed.values)

In [ ]:
data_train.CLAIM_TYPE.unique()

In [ ]:
list_cols=list(data_test_preprocessed)[:-1]
list_cols
#data_test_preprocessed['CLAIM_TYPE']=data_test_preprocessed.apply(lambda row: '-' if row['aggregated_class']==0 else  le.inverse_transform(modele2.predict(row[list_cols].values.reshape(1,-1))), axis=1)
#data_test_preprocessed['CLAIM_TYPE']=data_test_preprocessed.apply(lambda row: '-' if row['aggregated_class']==0 else  modele2.predict(row[list_cols].values.reshape(1,-1)), axis=1)
data_test_preprocessed_0=data_test_preprocessed[data_test_preprocessed['aggregated_class']==0]
data_test_preprocessed_1=data_test_preprocessed[data_test_preprocessed['aggregated_class']==1]



In [ ]:
data_test_preprocessed_0.head()

In [ ]:
data_test_preprocessed.head()

In [ ]:
data_test_preprocessed_1.head()

In [ ]:
data_test_preprocessed_0['CLAIM_TYPE']='-'
data_test_preprocessed_1['CLAIM_TYPE']=modele2.predict(data_test_preprocessed_1.drop(['aggregated_class'],axis=1).values)

In [ ]:

data_test_preprocessed_1['CLAIM_TYPE']=le.inverse_transform(data_test_preprocessed_1['CLAIM_TYPE'])

In [ ]:
data_test_preprocessed_1.head()

In [ ]:
data_test_preprocessed_final=pd.concat([data_test_preprocessed_0,data_test_preprocessed_1],axis=0)

In [ ]:
data_test_preprocessed_final.head()

In [ ]:
data_test_preprocessed_final=data_test_preprocessed_final.sort_index()

In [ ]:
data_test_preprocessed_final.head(5)

In [ ]:
data_test.head(5)

In [ ]:
y_pred=data_test_preprocessed_final.CLAIM_TYPE.values
y_pred

In [ ]:
prediction=pd.DataFrame({'a':data_test.ID,'b':y_pred})
prediction.columns = ['ID','CLAIM_TYPE']

In [ ]:
prediction.to_csv('prediction_Rakuten_All_Categorical_Corrected.csv',index=False,columns=['ID','CLAIM_TYPE'])

In [ ]:
pd.read_csv('prediction_Rakuten_All_Categorical_Corrected.csv')

In [ ]:
data_train.CLAIM_TYPE.value_counts()/len(data_train.CLAIM_TYPE)#ratio des classes en presence

In [ ]:
prediction.CLAIM_TYPE.value_counts()/len(prediction.CLAIM_TYPE)#ratio des classes en presence